<a href="https://colab.research.google.com/github/nerobite/neural_networks/blob/main/Hybrid_recommender_systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808330 sha256=22f177da57ce05c68ba98bc1fd0cd21f50a3ad47301db5a2fcfa54c0eb541006
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [ ]:
import pandas as pd
import numpy as np

from lightfm.datasets import fetch_movielens
from tqdm import tqdm

In [ ]:
movielens = fetch_movielens()
movielens

{'train': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 90570 stored elements in COOrdinate format>,
 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 9430 stored elements in COOrdinate format>,
 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
 	with 1682 stored elements in Compressed Sparse Row format>,
 'item_feature_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1'),
 'item_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1')}

In [ ]:
for key, value in movielens.items():
    print(key, type(value), value.shape)

train <class 'scipy.sparse._coo.coo_matrix'> (943, 1682)
test <class 'scipy.sparse._coo.coo_matrix'> (943, 1682)
item_features <class 'scipy.sparse._csr.csr_matrix'> (1682, 1682)
item_feature_labels <class 'numpy.ndarray'> (1682,)
item_labels <class 'numpy.ndarray'> (1682,)


In [ ]:
train = movielens['train']
test = movielens['test']

In [ ]:
from lightfm import LightFM

from lightfm.evaluation import auc_score


In [ ]:
featers = movielens['item_features']

In [ ]:
model = LightFM()
model.fit(train, epochs=10)

train_auc_score = auc_score(model, train, item_features=featers).mean()
test_auc_score = auc_score(model, test, train_interactions=train, item_features=featers).mean()

print('AUC Score: train %.2f, test %.2f.' % (train_auc_score, test_auc_score))

AUC Score: train 0.87, test 0.86.


In [ ]:
def sample_recommendation(model, data, user_ids):
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()
                          [user_id].indices]

        scores = model.predict(user_id, np.arange(n_items))

        top_items = data['item_labels'][np.argsort(-scores)]

        print("User %s" % user_id)
        print("     Known positives:")

        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")

        for x in top_items[:3]:
            print("        %s" % x)

In [ ]:
sample_recommendation(model, movielens, [1, 2, 5])

User 1
     Known positives:
        T
        R
        P
     Recommended:
        S
        F
        C
User 2
     Known positives:
        R
        C
        E
     Recommended:
        S
        C
        E
User 5
     Known positives:
        T
        T
        B
     Recommended:
        S
        F
        R
